In [ ]:
!pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=e203738f01fa4ca9bcd1619cf6c9b30dc77e1bd82075f3da8ab33d263ec730e4
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torch.utils.tensorboard import SummaryWriter
from PIL import UnidentifiedImageError
import os
from efficientnet_pytorch import EfficientNet

In [ ]:
import os
from os import walk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


In [ ]:
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn import Linear
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from torch.nn import Conv2d
from torch.nn import BatchNorm2d
from torch.nn import MaxPool2d
from torch.nn import Dropout
from torch.nn import Sequential
import torchvision.models as models



In [ ]:
try:
    import torchbearer
except:
    !pip install -q torchbearer
    import torchbearer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
try:
    from efficientnet_pytorch import EfficientNet
except:
    !pip install efficientnet_pytorch
    from efficientnet_pytorch import EfficientNet

**Preparing the trainer**

In [ ]:
import torchbearer
from torchbearer.callbacks import imaging

inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
    std=[1/0.229, 1/0.224, 1/0.255]
)

make_grid = imaging.MakeGrid(torchbearer.INPUT, num_images=64, nrow=8, transform=inv_normalize)
make_grid = make_grid.on_test().to_pyplot().to_file('sample.png')

In [ ]:
import torchbearer
from torchbearer import Trial

# If you have tensorboardX installed then write to tensorboard, else don't
import sys
if 'tensorboardX' in sys.modules:
  import tensorboardX
  from torchbearer.callbacks import TensorBoard
  callbacks = [TensorBoard(write_batch_metrics=True)]
else:
  callbacks = []

callbacks.append(make_grid)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Arrenging the dataset loader**

In [ ]:
HP = {
    'epochs': 25,
    'batch_size': 32,
    'learning_rate': 1e-3,
    'momentum': 0.9,
    'test_size': 0.05,
    'seed': 1
}

In [ ]:
torch.manual_seed(HP['seed'])
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.benchmark = True
print(f'using {device} device')

using cuda device


In [ ]:

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torch.utils.tensorboard import SummaryWriter
from PIL import UnidentifiedImageError
import os

In [ ]:
import pandas as pd


dataset_file = '/content/drive/MyDrive/CSV file of rice disease/rice_shuffled2.csv'




In [ ]:



df = pd.read_csv(dataset_file)


print(f'count: {len(df)} \n')
df.head(5)

count: 17929 



,Unnamed: 0,Image Location,Category No,Category
0,17267,/content/drive/MyDrive/paddy/Paddy main/normal...,9,normal
1,6088,/content/drive/MyDrive/paddy/Paddy main/hispa/...,4,hispa
2,4984,/content/drive/MyDrive/paddy/Paddy main/hispa/...,4,hispa
3,3770,/content/drive/MyDrive/paddy/Paddy main/brown_...,3,brown_spot
4,1069,/content/drive/MyDrive/paddy/Paddy main/dead_h...,1,dead_heart


In [ ]:
train_df, test_df = train_test_split(df, test_size=HP['test_size'])
print(f'train len: {len(train_df)}, test len: {len(test_df)}')

train len: 17032, test len: 897


In [ ]:
train_df['Category'].value_counts()

normal                      3192
blast                       3023
hispa                       2537
brown_spot                  2441
bacterial_leaf_blight       1721
tungro                      1460
dead_heart                  1379
downy_mildew                 592
bacterial_leaf_streak        366
bacterial_panicle_blight     321
Name: Category, dtype: int64

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomChoice([
        transforms.Pad(padding=10),
        transforms.CenterCrop(480),
        transforms.RandomRotation(60),
        transforms.CenterCrop((576,432)),
        transforms.ColorJitter(
            brightness=0.1,
            contrast=0.1,
            saturation=0.1,
            hue=0.1
        )
    ]),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

class PaddyDataset(Dataset):
    def __init__(self, df, transforms):
        self.df = df

        self.transforms = transforms
        #self.df = self.df.values.tolist()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        image = Image.open(self.df.iloc[idx]['Image Location']).convert('RGB')
        image = self.transforms(image)
        idx =self.df.iloc[idx]['Category No']

        return image, int(idx)


train_dataset = PaddyDataset(train_df, train_transform)
test_dataset = PaddyDataset(test_df, test_transform)
#test_dataloader = DataLoader(test_dataset, batch_size=HP['batch_size'], shuffle=True, pin_memory=True)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=55, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=55, shuffle=True, pin_memory=True)

In [ ]:
import os

class PaddyDataset(Dataset):
    def __init__(self, df, transforms):
        self.df = df
        self.transforms = transforms
        self.valid_file_paths = self._get_valid_file_paths()

    def _get_valid_file_paths(self):
        valid_paths = []
        for idx, row in self.df.iterrows():
            image_path = row['Image Location']
            if os.path.exists(image_path):
                valid_paths.append(image_path)
            else:
                print(f"Invalid file path: {image_path}")
        return valid_paths

    def __len__(self):
        return len(self.valid_file_paths)

    def __getitem__(self, idx):
        image_path = self.valid_file_paths[idx]

        try:
            image = Image.open(image_path).convert('RGB')
            image = self.transforms(image)
            category_idx = self.df[self.df['Image Location'] == image_path]['Category No'].values[0]
            return image, int(category_idx)
        except Exception as e:
            # Handle other exceptions (e.g., corrupted images) here
            print(f"Error loading image at index {idx}: {e}")
            # Return None or a special token to indicate an error
            return None, -1


In [ ]:
# Filter out rows with invalid file paths directly from the DataFrame
train_df = train_df[train_df['Image Location'].apply(lambda x: os.path.exists(x))]
test_df = test_df[test_df['Image Location'].apply(lambda x: os.path.exists(x))]


In [ ]:
# Create the dataset with the valid file paths
train_dataset = PaddyDataset(train_df, train_transform)
test_dataset = PaddyDataset(test_df, test_transform)


In [ ]:
# Print the number of samples in your datasets
print(f"Number of samples in the train dataset: {len(train_dataset)}")
print(f"Number of samples in the test dataset: {len(test_dataset)}")


Number of samples in the train dataset: 17032
Number of samples in the test dataset: 897


**Trainer**

In [ ]:
def trainer(model,epochs,lr,device=device):
  loss = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  trial = Trial(model, optimizer,loss, metrics=['acc', 'loss'], callbacks=callbacks).to(device)
  trial.with_generators(train_generator=train_dataloader, val_generator=test_dataloader)
  history = trial.run(epochs=epochs, verbose=2)
  return history

**Save model**

In [ ]:
import os
def save_model(model,savefile_Name):
  checkpoint = {'state_dict': model.state_dict()}
  torch.save(checkpoint, os.path.join('gdrive/MyDrive/',savefile_Name))

**model**

In [ ]:
!pip install torch-intermediate-layer-getter

  Preparing metadata (setup.py) ... done
  Created wheel for torch-intermediate-layer-getter: filename=torch_intermediate_layer_getter-0.1.post1-py3-none-any.whl size=3705 sha256=f2baf50d68f5f1d6e78fbbb64924576a2407518276df9a4635b0dd5451043b17
  Stored in directory: /root/.cache/pip/wheels/6a/11/c0/30d81aa26172d10d68ffaf352b0762eb9fe0a5f5dcf3de63e0
Successfully built torch-intermediate-layer-getter


In [ ]:
from torch_intermediate_layer_getter import IntermediateLayerGetter as MidGetter

In [ ]:
!pip install efficientnet-pytorch
from efficientnet_pytorch import EfficientNet

In [ ]:
# Define the EfficientnetB2 model without loading weights
class EfficientnetB2(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        # Declare the EfficientNet model without loading weights
        self.model = EfficientNet.from_pretrained('efficientnet-b2', num_classes=num_classes)

    def forward(self, x):
        out = self.model(x)
        return out

In [ ]:
efficientnet_b2 = EfficientnetB2(num_classes=10)  # Specify the number of output classes
efficientnet_b2.threshold = 0.33

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b2-8bb594d6.pth
100%|██████████| 35.1M/35.1M [00:00<00:00, 74.1MB/s]


Loaded pretrained weights for efficientnet-b2


In [ ]:
efficientnet_b2.load_state_dict(torch.load('/content/drive/MyDrive/CSV file of rice disease/efficientnet_b2 (3).pth'))


<All keys matched successfully>

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary

# Assuming you have already declared the 'efficientnet_b2' model
efficientnet_b2.to('cuda')  # Move the model to CUDA if available

# Display the summary of the model
summary(efficientnet_b2, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 225, 225]               0
Conv2dStaticSamePadding-2         [-1, 32, 112, 112]             864
       BatchNorm2d-3         [-1, 32, 112, 112]              64
MemoryEfficientSwish-4         [-1, 32, 112, 112]               0
         ZeroPad2d-5         [-1, 32, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 32, 112, 112]             288
       BatchNorm2d-7         [-1, 32, 112, 112]              64
MemoryEfficientSwish-8         [-1, 32, 112, 112]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!pip install torchbearer

In [ ]:
from torchbearer import Trial

In [ ]:
save=torchbearer.callbacks.checkpointers.Best(filepath='efficientnet_b2_0.33_2.pth', monitor='val_loss', mode='auto')
#save=save_model(efficientnet_b2,'efficientnet_b2_0.33_2.pth')
callbacks.append(save)

In [42]:

#Uncomment if you want to
#efficientnetb2_vit.to('cuda')
efficientnet_b2.train()

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(efficientnet_b2.parameters(), lr=3e-10)
trial = Trial(efficientnet_b2, optimizer,loss, metrics=['acc', 'loss']).to(device)
trial.with_generators(train_generator=train_dataloader, val_generator=test_dataloader)
history = trial.run(epochs=1, verbose=2)
#history=trainer(efficientnetb2_vit,6,5e-4)

0/1(t):   0%|          | 0/310 [00:00<?, ?it/s]

0/1(v):   0%|          | 0/17 [00:00<?, ?it/s]

In [43]:
torch.save(efficientnet_b2.state_dict(), 'efficientnet_b2.pth')